In [1]:
def str_to_boolean(bool_value_str):
    """Converts string indicators of True/False to boolean type. 
    Used for conversion of command line arguments to correct type."""
    input_str_lower = bool_value_str.lower()  # Convert input string to lowercase
    if input_str_lower == "true":
        return True
    elif input_str_lower == "false":
        return False
    else:
        raise ValueError('Boolean values must be "true" or "false".')

In [5]:
str_to_boolean("falsdse")

ValueError: Boolean values must be "true" or "false".

In [11]:
'''
Author: Alexander Macejovsky, Stepan Pardubicky
Date: 29.2.2024

This is script for using tools for automated classification and correction of errors in TS.


Parameters:
The scripts requires indicators of whether to print debug messages and whether to only print parameters, and parameters 
denoting names of input and output dfs0 files. 
Additionally, parameters indicating characteristics of concerned TS and customized parameters for classification and 
correction of errors can be provided in the following order and format:
    TS_characteristics (dictionary): {"main_var": "ts_var_name", "start_date":"dd-mm-yyyy", "end_date":"dd-mm-yyyy", 
                                    "periodicity":"n(no_of_minutes)T", "check_per":boolean, "date_col":"name_of_date_column"}
    Classification_params (dictionary): {"W_0":numeric, "tol_const":numeric,
                                         "c_1":numeric, "W_1":numeric,
                                         "c_2":numeric, "W_2":numeric, "p_1":numeric,
                                         "W_3":numeric, "p_2":numeric,
                                         "tol_vol_1":numeric, "tol_vol_2":numeric,
                                         "tol_rain_1":numeric, "tol_rain_2":numeric,
                                         "volatile_diffs":boolean,
                                         "num_back":numeric, "num_fol":numeric,
                                         "fol_tresh":numeric, "W_4":numeric, "c_3":numeric}
    Correction_params (dictionary): {"outliers_window":numeric, "volatility_window":numeric}

Not specifying an argument leads to use of its default values. See documentation of TS_Class, TS_Class.classify and 
correct_data in DHI_functions module and other relevant documentation for more information on these parameters.


Output:
The output dataset contains, in addition to the values of the input dataset, new columns with .... TO DO


Use examples when used from the command line:
    python 1 0 DHI_script.py 'input_file.dfs0' 'output_file.dfs0' '{"main_var": "rychlost", "periodicity": "4T"}' '{"c_2": 2, "volatile_diffs": "True"}' '{"volatility_window": 2}'
    python 1 0 DHI_script.py 'input_file.dfs0' 'output_file.dfs0' '{}' '{"p_2": 1, "volatile_diffs": "False"}' 
    python 0 0 DHI_script.py 'input_file.dfs0' 'output_file.dfs0'

In the first example, name of the variable of concern is specified as "rychlost" and periodicity of observations 
is specified as one observation per 4 minutes; classification parameter c_2 is set to 2 and volatile_diffs is 
set to True; correction parameter volatility_window is set to 2; other TS characteristics, classification and 
correction parameters are left at their default values.
In the second example, all TS characteristics are left at their default values; classification parameter p_2 is 
set to 1 and volatile_diffs is set to False; and all other classification parameters as well as all correction 
parameters are left at their default values.
Example 3 uses the script with all default values.


-- to do--
Can we read main_var directly from the input dataset?

--
When using this sample for your own script, fill in names of parameters wanted in the first paragraph and
change the body of the script as needed.
Do not change other parts of script, unless you know certain what you are doing :)

Keep in mind that pip.exe must be included in environmental variabl PATH in the version corresponding to the python
executor used, for installing any packages including mikeio. --
'''

############ Parameters:
parameterNames = ["TS_characteristics", "Classification_params", "Correction_params"]   ##
############ End of parameters

# TO DO:
* make sure input dictionaries are correctly transformed to Python dictionaries, inlcuding preservation of numeric, string and boolean types
* make sure length of args is correct, take care about cases when no dictionaries are provided
* perform classification and correction
* select what should be included in the output
* can we read main_var directly from the data?

* finish underlying functions and methods, add documentation to the most relevant ones

In [ ]:
import sys, subprocess
from DHI_functions import *   ##

# read arguments
args = sys.argv

debug = int(args[1])
onlyParameters = int(args[2])

# return parameter names (when GND button "Get parameters" clicked)
if (onlyParameters == 1):
    for p in parameterNames:
        print(p)
    sys.exit()

# when executing script
if onlyParameters==0:
    try:
        import mikeio
    except ImportError as e:
        if debug:
            print("Package mikeio not installed. Installing...")
        subprocess.check_call("pip install mikeio")
        import mikeio

    # read arguments
    inputFileName = args[3]
    outputFileName = args[4]
    params = args[5:]

    if debug:
        print("Python engine:")
        print(sys.executable)
        print("Running script:")
        print(args[0])





if debug == 1:
    if len(parameterNames) == len(params):
        print("\nParameters used:")
        for i in range(0,len(parameterNames)):
            print(parameterNames[i] + " = " + params[i])
    else:
        raise ValueError("Length of parameters and parameter names differs!")

############ BODY (edit script as needed):
    
# read input dfs0
inputDfs = mikeio.Dfs0(inputFileName)

# read input item info
inputItemInfo = inputDfs.items[0]
if debug:
    print("\nInput item info")
    print(inputItemInfo)

# read type and units of input TS
inputType = inputDfs.items[0].type
inputUnits = inputDfs.items[0].unit

# print header
if debug:
    print("\nInput TS type: ")
    print(inputType)    
    print("\nInput TS unit: ")
    print(inputUnits)

# extract data to dataframe
inputDataframe = inputDfs.to_dataframe()

if debug:
    print("\nInput TS preview:")
    print(inputDataframe.head())


### create output TS
outputDataframe = inputDataframe * float(params[0]) + float(params[1])

# print header
if debug:
    print("\nOutput TS preview:")
    print(outputDataframe.head())

# Item info
# outputItemInfo = mikeio.ItemInfo("Relative_humidity", inputType, inputUnits)
outputItemInfo = inputItemInfo

# output item info:
if debug:
    print("\nOutput item info:")
    print(outputItemInfo)


############ End of BODY

#write output dfs0
outputDfs = mikeio.Dfs0()
outputDfs.from_dataframe(outputDataframe, outputFileName, items=[outputItemInfo])


print ("Succesfully finished")